In [ ]:
!pip install tensorflow streamlit pyngrok Pillow

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image
import os

In [ ]:
pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [ ]:
import os

base_dir = os.path.expanduser("C:/Users/adhar/Downloads/eyenova/kayser_fleischer")
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")
test_dir = os.path.join(base_dir, "test")

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def preprocess_input(image):
    try:
        return tf.image.resize(image, (224, 224)) / 255.0
    except Exception as e:
        print(f"Error processing image: {e}")
        return None

train_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode= 'nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input)


# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # binary classification
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
print(train_generator.samples)
print(val_generator.samples)
print(test_generator.samples)

steps_per_epoch = train_generator.samples // train_generator.batch_size

validation_steps = val_generator.samples // val_generator.batch_size

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    class_weight={1:31.5, 0:1}
)

In [ ]:
model.save('kfr_model.h5')

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
#GLAUCOMA CLASSIFICATION

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

glaucoma_base_dir = 'C:/Users/adhar/Downloads/eyenova/glaucoma'
glaucoma_train_dir = os.path.join(glaucoma_base_dir, "train")
glaucoma_val_dir = os.path.join(glaucoma_base_dir, "val")
glaucoma_test_dir = os.path.join(glaucoma_base_dir, "test")

glaucoma_train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

glaucoma_val_datagen = ImageDataGenerator(rescale=1./255)
glaucoma_test_datagen = ImageDataGenerator(rescale=1./255)
glaucoma_train_generator = glaucoma_train_datagen.flow_from_directory(
    glaucoma_train_dir, target_size=(224, 224), batch_size=32, class_mode='binary'
)
glaucoma_val_generator = glaucoma_val_datagen.flow_from_directory(
    glaucoma_val_dir, target_size=(224, 224), batch_size=32, class_mode='binary'
)
glaucoma_test_generator = glaucoma_test_datagen.flow_from_directory(
    glaucoma_test_dir, target_size=(224, 224), batch_size=32, class_mode='binary',
    shuffle=False
)

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

glaucoma_model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


glaucoma_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

glaucoma_model.summary()

In [ ]:
history = glaucoma_model.fit(
    glaucoma_train_generator,
    validation_data=glaucoma_val_generator,
    epochs=10,
    steps_per_epoch=glaucoma_train_generator.samples // 32,
    validation_steps=glaucoma_val_generator.samples // 32
)

In [ ]:
glaucoma_model.save('glaucoma_model.h5')

In [ ]:
loss, accuracy = model.evaluate(glaucoma_test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
import os
print(os.getcwd())